# AIMO Prize 3 - OOP Baseline Submission (v3)

This notebook demonstrates how to use the Object-Oriented Baseline with **Few-Shot Prompting**, **Majority Voting**, and **Parquet Submission**.

## Setup Instructions
1. **Add Utility Script**: Upload `src/kaggle_baseline.py` as a Dataset (e.g., named `aimo-pp3-modules`).
2. **Add Model**: Search for and add the model `Qwen/Qwen2.5-Math-7B-Instruct` (or a quantized version).
3. **Run**: Execute the cells below.

In [ ]:
import sys
import os

# 1. Import the OOP Baseline Module
sys.path.append('/kaggle/input/aimo-pp3-modules')
sys.path.append(os.path.join(os.getcwd(), 'src')) 

try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM, LocalVLLM
    print("✅ Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"❌ Import Failed: {e}. Make sure the 'aimo-pp3-modules' dataset is attached.")

In [ ]:
# 2. Configure Environment & Model
config = CompetitionConfig()
config.n_repetitions = 1 # Set to 3-5 for Majority Voting (requires GPU)

print(f"Environment: {'Kaggle' if config.is_kaggle else 'Local'}")
print(f"Model Path: {config.model_path}")

if config.is_kaggle and os.path.exists(config.model_path):
    # Uncomment for Real Model (vLLM)
    # llm = LocalVLLM(config.model_path)
    print("⚠️ Using MockLLM. Uncomment LocalVLLM lines to use the real model.")
    llm = MockLLM()
else:
    llm = MockLLM()

solver = AIMSolver(config, llm)

## Submission Loop (Parquet)

In [ ]:
import pandas as pd

# Attempt to import the competition API
# Try standard 'aimo_3_gateway' or fallbacks
try:
    import aimo_3_gateway as aimo
except ImportError:
    try:
        import aimo_3_inference_server as aimo
    except ImportError:
        aimo = None

if aimo:
    try:
        env = aimo.make_env()
        iter_test = env.iter_test()
        print("✅ Connected to AIMO Gateway.")
        
        all_preds = []
        
        for test, sample_submission in iter_test:
            for index, row in test.iterrows():
                result = solver.solve(row['problem'])
                sample_submission.loc[index, 'answer'] = result
            
            # Accumulate for Parquet
            all_preds.append(sample_submission.copy())
            
            # Submit to API (it might write CSV or do nothing visible)
            env.predict(sample_submission)
            print(f"Submitted batch.")
        
        # Explicitly write submission.parquet
        if all_preds:
            final_df = pd.concat(all_preds)
            final_df.to_parquet('submission.parquet', index=False)
            print("✅ submission.parquet created.")
            
    except Exception as e:
        print(f"❌ Submission Error: {e}")
else:
    print("⚠️ AIMO API not found. Skipping submission loop (Local Mode).")
    # Create dummy parquet for verification if running locally
    if not config.is_kaggle:
        pd.DataFrame({'id': ['test_id'], 'answer': [42]}).to_parquet('submission.parquet')
        print("Created dummy submission.parquet for local test.")